## SSL method: InfoNCE (Multimodal)

In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../") # Add the parent directory to the path
from utils.reproducibility import set_seed
set_seed(1)
import torch
import os
from utils.get_data import AVMNISTDataModule
from utils.plots_trials import load_all_versions, save_versions_to_csv, plot_loss
from utils.visualisations import pca_plot_dataloaders, pca_plot_multiclass, visualize_prediction_matrix, tsne_plot_multiclass
from training_structures.dino_train import train_downstream, train_knn_classifier, compute_classification_metrics
from training_structures.ssl_train import train_and_evaluate_ssl
from info_nce import MultiModalInfoNCELightning
current_path = os.getcwd()
root_path = "../../"
data_dir=f'{root_path}/data/avmnist/'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
module = MultiModalInfoNCELightning(
    projection_dim=256,
    output_dim=256,
    learning_rate=0.001,
    num_epochs=100,
    use_mixed_precision=True
)
model = module.model.to(device)

In [94]:
def compute_train_results(pretrained_model, model_path, model_name, traindata, validdata, testdata):
    """
    Save Plot results (e.g. pca for frozen encoders) and 
    Accuracies (for downstream tasks (KNN and MLP on 10 epochs))
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pretrained_model.to(device)

    knn_model, knn_accuracy = train_knn_classifier(pretrained_model, traindata, testdata, n_neighbors=5, is_dino_based=False)
    classifier = train_downstream(
        pretrained_model,
        traindata,
        validdata,
        testdata,
        num_epochs=10,
        device=device,
        save_path=f'{model_path}/downstream/{model_name}.pt',
        train_log_path=f'{model_path}/downstream/{model_name}_train_log.csv',
        test_log_path=f'{model_path}/downstream/{model_name}_test_log.csv',
        is_dino_based=False,
    )
    mlp_accuracy = compute_classification_metrics(classifier, testdata, device)['accuracy']

    return knn_accuracy, mlp_accuracy, classifier

In [95]:
def visualize_results(model_path, model_name, encoder, classifier, testdata):
    pca_plot_path = os.path.join(model_path, f'{model_name}_pca_plots')
    confusion_matrix_path = os.path.join(model_path, f'{model_name}_confusion_matrix')
    log_path = os.path.join(model_path, f'logs')
    
    _ = pca_plot_dataloaders(encoder, testdata, selected_digits=[5, 8], 
                             dirpath=pca_plot_path, show_plots=False, is_dino_based=False)
    _ = pca_plot_multiclass(encoder, testdata, selected_digits=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                            dirpath=pca_plot_path, show_plots=False, is_dino_based=False)
    _ = visualize_prediction_matrix(classifier, testdata, dirpath=confusion_matrix_path, show_plots=False)
    _ = tsne_plot_multiclass(encoder, testdata, selected_digits=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                            dirpath=pca_plot_path, show_plots=False, random_seed=0, is_dino_based=False)
    
    metrics_df = load_all_versions(log_path)
    save_versions_to_csv(metrics_df, model_path)
    plot_loss(metrics_df, plot_dir=model_path)

In [ ]:
data_module = AVMNISTDataModule(data_dir=data_dir, batch_size=128, num_workers=0)
num_epochs = 100
model_dir, model_name = f"{current_path}", "info_nce_model"
module = MultiModalInfoNCELightning(projection_dim=256, output_dim=256, learning_rate=0.001, 
                                    num_epochs=num_epochs, use_mixed_precision=True)
seeds=[1, 2, 3]
input_data=[
    (torch.randn(1, 1, 28, 28),
    torch.randn(1, 1, 112, 112),  
    torch.tensor([1]))      
]

train_and_evaluate_ssl(
    module=module,
    train_data_module=data_module,
    test_data_module=data_module,
    model_dir=model_dir,
    model_name=model_name,
    input_data=input_data,
    seeds=seeds,
    num_epochs=num_epochs,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | InfoNCEModel | 1.3 M  | train
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.092     Total estimated model params size (MB)
55        Modules in train mode
0         Modules in eval mode


Model GFLOPs: 0.19 GFLOPs
Model Parameters: 1.27 M parameters
Training with seed 1...
Epoch 99: 100%|██████████| 430/430 [00:25<00:00, 17.05it/s, v_num=eed1, train_loss_step=0.344, train_loss_epoch=0.440]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 430/430 [00:25<00:00, 17.00it/s, v_num=eed1, train_loss_step=0.344, train_loss_epoch=0.440]
Extracting training features...


100%|██████████| 430/430 [00:15<00:00, 27.69it/s]


...

Visualizing digits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


c:\Users\Ward\Desktop\vub-github\Thesis-project\AVMNIST_Experiments\other_ssl\info_nce\../..\utils\visualisations.py:750: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


100%|██████████| 79/79 [00:01<00:00, 56.51it/s]


Visualizing digits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Applying t-SNE...


c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\sklearn\manifold\_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
